In [ ]:
import tqdm, re, pandas as pd, random, pprint
import matplotlib.pyplot as plt
import pickle as pkl

In [ ]:
from wordcloud import WordCloud, STOPWORDS
stopwords= set(STOPWORDS)

In [ ]:
def get_memory():
    status= open('/proc/self/status').read()
    memuse= int(re.findall('VmRSS:\s*(\d+)\skB', status)[0]) / 1024 ** 2
    print('Resident memory: %.2f GB' % memuse)

In [ ]:
get_memory()

In [ ]:
def loadPickle(name):
    return pd.read_pickle(f'../data/{name}')
def loadTxt(name):
    return [i.strip() for i in open(f'../data/{name}', 'r').readlines()]
def dumpPickle(df, name):
    df.to_pickle(f'../data/{name}')

In [ ]:
restoredUsersList= set()
for i in loadTxt('unsuspended_8_Oct.txt'):
    restoredUsersList.add(i)

In [ ]:
RegB= loadPickle('Regular_Before_py37.pkl')
RegA= loadPickle('Regular_After_py37.pkl')
# Find the common users
regularBefore= RegB[RegB['user.id_str'].isin(RegA['user.id_str'])].copy()
regularAfter= RegA[RegA['user.id_str'].isin(RegB['user.id_str'])].copy()
# Set the type of user
regularBefore['typeOfUser']= 'regularBefore'
regularAfter['typeOfUser']= 'regularAfter'

In [ ]:
ResB= loadPickle('Restored_Before_py37.pkl')
ResA= loadPickle('Restored_After_py37.pkl')
# Find he common users
restoredBefore= ResB[ResB['user.id_str'].isin(ResA['user.id_str'])==True].copy()
restoredAfter= ResA[ResA['user.id_str'].isin(ResB['user.id_str'])==True].copy()
# Set the type of user
restoredBefore['typeOfUser']= 'restoredBefore'
restoredAfter['typeOfUser']= 'restoredAfter'

In [ ]:
pklDict= {
    'restoredBefore': restoredBefore,
    'restoredAfter': restoredAfter,
    'regularBefore': regularBefore,
    'regularAfter': regularAfter,
}
ResA= None
ResB= None
RegA= None
RegB= None
regularBefore= None
regularAfter= None
restoredBefore= None
restoredAfter= None

In [ ]:
# Define the colors for grayscale
def grey_color_func(word, font_size, position, orientation, random_state=None, **kwargs):
   return "hsl(0, 0%%, %d%%)" % random.randint(15, 45)

In [ ]:
# Preprocess the data (remove hashtag, links and mentions from the text)
def preprocess(data):
    comment_words= ''
    for i in tqdm.tqdm(data):
        if isinstance(i, str):
            tokens= re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", i).split()
            for i in range(len(tokens)):
                tokens[i]= tokens[i].lower() 
            comment_words+= " ".join([i for i in tokens if len(i)>3]) + " "
    return comment_words

In [ ]:
# Plotting the wordcloud
def plotWordCloud(data):

    # Configuring the WordCloud api
    wordcloud= WordCloud(
        collocations= False, # To remove duplicate entries
        width= 1000, 
        height= 1000,
        background_color= '#ffffff',
        stopwords= stopwords, 
        min_font_size= 10
    ).generate(preprocess(data))
    wordcloud.recolor(color_func= grey_color_func)
    
    # plot the WordCloud
    plt.figure(figsize= (8, 8), facecolor= None)
    plt.imshow(wordcloud) 
    plt.axis("off") 
    plt.tight_layout(pad= 0)
    plt.show()

## What restored before were talking about

In [ ]:
plotWordCloud(list(pklDict['restoredBefore']['full_text']))

## What restored after are talking about

In [ ]:
plotWordCloud(list(pklDict['restoredAfter']['full_text']))

## What regular before were talking about

In [ ]:
plotWordCloud(list(pklDict['regularBefore']['full_text']))

## What regular after are talking about

In [ ]:
plotWordCloud(list(pklDict['regularAfter']['full_text']))

In [ ]:
normal_tweets= []
with open(f'../data/all_tweets_normal_jan3_2021.pickle', 'rb') as f:
    try:
        while True:
            temp= pkl.load(f)
            for i in tqdm.tqdm(temp):
                if i.get('retweeted_status'):
                    if i['retweeted_status'].get('extended_tweet'):
                        if i['retweeted_status']['extended_tweet'].get('full_text'):
                            normal_tweets.append(i['retweeted_status']['extended_tweet']['full_text'])
                        else:
                            print('--Here--1--')
                            pprint.pprint(i)
                            break
                    else:
                        if i['retweeted_status'].get('full_text'):
                            normal_tweets.append(i['retweeted_status']['full_text'])
                        elif i['retweeted_status'].get('text'):
                            normal_tweets.append(i['retweeted_status']['text'])
                        else:
                            print('--Here--2--')
                            pprint.pprint(i)
                            break
                elif i.get('full_text'):
                    normal_tweets.append(i['full_text'])
                elif i.get('text'):
                    normal_tweets.append(i['text'])
                else:
                    print('--Here--3--')
                    pprint.pprint(i)
                    break
    except EOFError:
        pass

In [ ]:
plotWordCloud(normal_tweets)

In [ ]:
suspended_tweets= []
restored_tweets= []
with open(f'../data/all_tweets_suspended_jan3_2021.pickle', 'rb') as f:
    try:
        while True:
            temp= pkl.load(f)
            for i in tqdm.tqdm(temp):
                if i.get('retweeted_status'):
                    if i['retweeted_status'].get('extended_tweet'):
                        if i['retweeted_status']['extended_tweet'].get('full_text'):
                            if i['retweeted_status']['user']['id_str'] in restoredUsersList:
                                restored_tweets.append(i['retweeted_status']['extended_tweet']['full_text'])
                            else:
                                suspended_tweets.append(i['retweeted_status']['extended_tweet']['full_text'])
                        else:
                            print('--Here--1--')
                            pprint.pprint(i)
                            break
                    else:
                        if i['retweeted_status'].get('full_text'):
                            if i['retweeted_status']['user']['id_str'] in restoredUsersList:
                                restored_tweets.append(i['retweeted_status']['full_text'])
                            else:
                                suspended_tweets.append(i['retweeted_status']['full_text'])
                        elif i['retweeted_status'].get('text'):
                            if i['retweeted_status']['user']['id_str'] in restoredUsersList:
                                restored_tweets.append(i['retweeted_status']['text'])
                            else:
                                suspended_tweets.append(i['retweeted_status']['text'])
                        else:
                            print('--Here--2--')
                            pprint.pprint(i)
                            break
                elif i.get('full_text'):
                    if i['user']['id_str'] in restoredUsersList:
                        restored_tweets.append(i['full_text'])
                    else:
                        suspended_tweets.append(i['full_text'])
                elif i.get('text'):
                    if i['user']['id_str'] in restoredUsersList:
                        restored_tweets.append(i['text'])
                    else:
                        suspended_tweets.append(i['text'])
                else:
                    print('--Here--3--')
                    pprint.pprint(i)
                    break
    except EOFError:
        pass

In [ ]:
plotWordCloud(suspended_tweets)

In [ ]:
plotWordCloud(restored_tweets)